In [5]:
import pathview
import json
from pathsim.blocks import Scope
import matplotlib.pyplot as plt
import pandas as pd

# read graph data from a JSON file
with open("ARC_with_single_GLC.json", "r") as f:
    graph_data = json.load(f)

model, duration = pathview.make_pathsim_model(graph_data)

model.run(1 * 24 * 3600)


20:45:53 - INFO - LOGGING (log: True)
20:45:53 - INFO - BLOCKS (total: 36, dynamic: 10, static: 26, eventful: 1)
20:45:53 - INFO - GRAPH (nodes: 36, edges: 52, alg. depth: 6, loop depth: 0, runtime: 0.086ms)
20:45:53 - INFO - STARTING -> TRANSIENT (Duration: 86400.00s)
20:45:54 - INFO - --------------------   1% | 0.3s<15.9s | 25.4 it/s
20:45:55 - INFO - #-------------------   5% | 1.3s<18.9s | 20.6 it/s
20:45:56 - INFO - #-------------------   8% | 2.4s<38.0s | 18.2 it/s
20:45:57 - INFO - ##------------------  11% | 3.4s<44.7s | 21.2 it/s
20:45:58 - INFO - ##------------------  14% | 4.4s<37.3s | 19.2 it/s
20:45:59 - INFO - ###-----------------  16% | 5.5s<33.4s | 19.0 it/s
20:46:00 - INFO - ###-----------------  19% | 6.5s<26.5s | 20.7 it/s
20:46:00 - INFO - ####----------------  20% | 6.6s<18.3s | 21.1 it/s
20:46:01 - INFO - ####----------------  23% | 7.7s<21.3s | 22.7 it/s
20:46:02 - INFO - #####---------------  26% | 8.7s<19.1s | 23.7 it/s
20:46:03 - INFO - #####---------------  

{'total_steps': 736, 'successful_steps': 563, 'runtime_ms': 36205.13797500098}

In [6]:
import morethemes as mt
mt.set_theme("lighter")

In [ ]:
scopes = [block for block in model.blocks if isinstance(block, Scope)]

ifc_scope = scopes[1]
ofc_scope = scopes[0]
fusion_rate_scope = scopes[2]

for scope in [ofc_scope, ifc_scope]:
    sim_time, data = scope.read()
    for p, d in enumerate(data):
        lb = scope.labels[p] if p < len(scope.labels) else f"port {p}"
        if any([kw in lb.lower() for kw in ["blanket", "tes", "iss", "storage"]]):
            l, = plt.plot(sim_time / 3600 / 24, d, label=lb, linewidth=4)

            if "storage" in lb.lower():
                storage_colour = l.get_color()
            elif "blanket" in lb.lower():
                blanket_colour = l.get_color()

    plt.xlabel("Time (days)")
    plt.ylabel("inventory (kg)")
    plt.yscale("log")
    plt.xscale("log")
plt.ylim(1e-3, 1e2)
plt.xlim(1e-1, 1e3)

plt.legend()
plt.tight_layout()
plt.savefig("arc_with_single_GLC.pdf")
